# Quantitative Portfolio Optimization and Rebalancing

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [ ]:
assets = ['RELIANCE.NS', 'INFY.NS', 'HDFCBANK.NS', 'ICICIBANK.NS']
start_date = '2020-01-01'
end_date = '2024-12-31'

prices = yf.download(assets, start=start_date, end=end_date)['Adj Close']
returns = prices.pct_change().dropna()

In [ ]:
mean_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252

In [ ]:
def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.dot(weights, mean_returns)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return returns, volatility

def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.05):
    p_ret, p_vol = portfolio_performance(weights, mean_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / p_vol

num_assets = len(assets)
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for asset in range(num_assets))

initial_weights = num_assets * [1. / num_assets,]
opt_results = minimize(neg_sharpe_ratio, initial_weights,
                       args=(mean_returns, cov_matrix),
                       method='SLSQP', bounds=bounds, constraints=constraints)
optimal_weights = opt_results.x

In [ ]:
ret, vol = portfolio_performance(optimal_weights, mean_returns, cov_matrix)
sharpe = (ret - 0.05) / vol

print('Optimal Portfolio Weights:')
for i, asset in enumerate(assets):
    print(f'{asset}: {optimal_weights[i]:.2%}')

print(f'Expected Annual Return: {ret:.2%}')
print(f'Annual Volatility: {vol:.2%}')
print(f'Sharpe Ratio: {sharpe:.2f}')

In [ ]:
weights_df = pd.DataFrame(np.repeat([optimal_weights], len(returns), axis=0),
                          columns=assets, index=returns.index)

portfolio_returns = (returns * weights_df).sum(axis=1)
cumulative_returns = (1 + portfolio_returns).cumprod()

plt.figure(figsize=(10, 5))
plt.plot(cumulative_returns, label='Optimized Portfolio')
plt.title('Portfolio Cumulative Return (2020–2024)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
rebalance_window = 60
rebalance_dates = returns.index[::rebalance_window]
portfolio_values = [1]

for i in range(1, len(rebalance_dates)):
    sub_returns = returns.loc[rebalance_dates[i-1]:rebalance_dates[i]]
    mean_returns_sub = sub_returns.mean() * 252
    cov_sub = sub_returns.cov() * 252
    opt = minimize(neg_sharpe_ratio, initial_weights,
                   args=(mean_returns_sub, cov_sub),
                   method='SLSQP', bounds=bounds, constraints=constraints)
    w = opt.x
    growth = (1 + (sub_returns * w).sum(axis=1)).prod()
    portfolio_values.append(portfolio_values[-1] * growth)

plt.figure(figsize=(10, 5))
plt.plot(rebalance_dates, portfolio_values, marker='o', label='Rebalanced Portfolio')
plt.title('Portfolio Value with Periodic Rebalancing')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.legend()
plt.grid(True)
plt.show()